In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
import argparse
import psutil
import numpy as np
import logging
import numpy as np
import tensorflow as tf
import time
import glob
from tensorflow.python.framework import ops
import math
from dotenv import find_dotenv, load_dotenv
import ftplib

from sklearn import decomposition

nb_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'data')
if nb_dir not in sys.path:
    sys.path.insert(0, nb_dir)
print(sys.path)
import features_selection as fs
import make_dataset as md
import build_data as bd
import get_raw_data as grd

models_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'models')
if models_dir not in sys.path:
    sys.path.insert(0, models_dir)
import nn_real as nn

load_dotenv(find_dotenv())

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['/home/ubuntu/MLMortgage/src/data', '', '/home/ubuntu/src/cntk/bindings/python', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python36.zip', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/lib-dynload', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions', '/home/ubuntu/.ipython']


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
2018-11-27 15:34:36,525 - matplotlib - DEBUG - CACHEDIR=/home/ubuntu/.cache/matplotlib
2018-11-27 15:34:36,633 - matplotlib.font_manager - DEBUG - Using fontManager instance from /home/ubuntu/.cache/matplotlib/fontList.json
2018-11-27 15:34:38,405 - matplotlib.backends - DEBUG - backend module://ipykernel.pylab.backend_inline version unknown


True

2018-11-27 15:34:39,299 - matplotlib.backends - DEBUG - backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

RAW_DIR = os.path.join(Path(os.getcwd()).parents[0], 'data', 'raw') 
PRO_DIR = os.path.join(Path(os.getcwd()).parents[0], 'data', 'processed')
RANDOM_SEED = 123  # Set the seed to get reproducable results.
DT_FLOAT = np.float32
NP_FLOAT = np.dtype('float32')

train_dir = 'chuncks_random_c1millx2_train'
valid_dir = 'chuncks_random_c1millx2_valid'
test_dir = 'chuncks_random_c1millx2_test'
train_period=[121,323] #[121,279] #[121, 143] 
valid_period=[324,329] #[280,285] #[144, 147] 
test_period=[330,342] #[286,304] #[148, 155]
batch_size=50000


print(RAW_DIR, PRO_DIR)

/home/ubuntu/MLMortgage/data/raw /home/ubuntu/MLMortgage/data/processed


In [3]:
def get_files_dict(train_dir, valid_dir, test_dir):        
    ext = "*.h5"

    files_dict = {'train': glob.glob(os.path.join(PRO_DIR, train_dir, ext)), 
                  'valid': glob.glob(os.path.join(PRO_DIR, valid_dir, ext)), 
                  'test': glob.glob(os.path.join(PRO_DIR, test_dir, ext))}

    return files_dict

def architecture_settings(files_dict):
    architecture = {}    
    ok_inputs = True
    
    for key in files_dict.keys():
        total_records = 0
        for file in files_dict[key]:                                
            with pd.HDFStore(file) as dataset_file:
                if (ok_inputs): 
                    index_length = len(dataset_file.get_storer(key+'/features').attrs.data_columns)
                    architecture['n_input'] = dataset_file.get_storer(key+ '/features').ncols - index_length
                    architecture['n_classes'] = dataset_file.get_storer(key+'/labels').ncols - index_length
                    ok_inputs = False                
                total_records += dataset_file.get_storer(key + '/features').nrows
        architecture[key + '_num_examples'] = total_records                            
    
    architecture['total_num_examples'] = architecture['train_num_examples'] # 10000000
    return architecture

In [4]:
#To sum up the dataset per worker (assuming the same size of files per worker approximately):
files_dict = get_files_dict(train_dir, valid_dir, test_dir)
architecture = architecture_settings(files_dict)
print(architecture)

{'n_input': 258, 'n_classes': 7, 'train_num_examples': 26769588, 'valid_num_examples': 901868, 'test_num_examples': 0, 'total_num_examples': 26769588}


In [5]:
numeric_cols = ['MBA_DAYS_DELINQUENT', 'MBA_DAYS_DELINQUENT_NAN',
       'CURRENT_INTEREST_RATE', 'CURRENT_INTEREST_RATE_NAN', 'LOANAGE', 'LOANAGE_NAN',
       'CURRENT_BALANCE', 'CURRENT_BALANCE_NAN', 'SCHEDULED_PRINCIPAL',
       'SCHEDULED_PRINCIPAL_NAN', 'SCHEDULED_MONTHLY_PANDI',
       'SCHEDULED_MONTHLY_PANDI_NAN', 
       'LLMA2_CURRENT_INTEREST_SPREAD', 'LLMA2_CURRENT_INTEREST_SPREAD_NAN',  
       'LLMA2_C_IN_LAST_12_MONTHS',
       'LLMA2_30_IN_LAST_12_MONTHS', 'LLMA2_60_IN_LAST_12_MONTHS',
       'LLMA2_90_IN_LAST_12_MONTHS', 'LLMA2_FC_IN_LAST_12_MONTHS',
       'LLMA2_REO_IN_LAST_12_MONTHS', 'LLMA2_0_IN_LAST_12_MONTHS',
       'LLMA2_HIST_LAST_12_MONTHS_MIS', 
       'NUM_MODIF', 'NUM_MODIF_NAN', 'P_RATE_TO_MOD', 'P_RATE_TO_MOD_NAN', 'MOD_RATE',
       'MOD_RATE_NAN', 'DIF_RATE', 'DIF_RATE_NAN', 'P_MONTHLY_PAY',
       'P_MONTHLY_PAY_NAN', 'MOD_MONTHLY_PAY', 'MOD_MONTHLY_PAY_NAN',
       'DIF_MONTHLY_PAY', 'DIF_MONTHLY_PAY_NAN', 'CAPITALIZATION_AMT',
       'CAPITALIZATION_AMT_NAN', 'MORTGAGE_RATE', 'MORTGAGE_RATE_NAN',
       'FICO_SCORE_ORIGINATION', 'INITIAL_INTEREST_RATE', 'ORIGINAL_LTV',
       'ORIGINAL_BALANCE', 'BACKEND_RATIO', 'BACKEND_RATIO_NAN',
       'ORIGINAL_TERM', 'ORIGINAL_TERM_NAN', 'SALE_PRICE', 'SALE_PRICE_NAN', 	   
       'PREPAY_PENALTY_TERM', 'PREPAY_PENALTY_TERM_NAN', 
        'NUMBER_OF_UNITS', 'NUMBER_OF_UNITS_NAN', 'MARGIN',
       'MARGIN_NAN', 'PERIODIC_RATE_CAP', 'PERIODIC_RATE_CAP_NAN',
       'PERIODIC_RATE_FLOOR', 'PERIODIC_RATE_FLOOR_NAN', 'LIFETIME_RATE_CAP',
       'LIFETIME_RATE_CAP_NAN', 'LIFETIME_RATE_FLOOR',
       'LIFETIME_RATE_FLOOR_NAN', 'RATE_RESET_FREQUENCY',
       'RATE_RESET_FREQUENCY_NAN', 'PAY_RESET_FREQUENCY',
       'PAY_RESET_FREQUENCY_NAN', 'FIRST_RATE_RESET_PERIOD',
       'FIRST_RATE_RESET_PERIOD_NAN', 	           
       'LLMA2_ORIG_RATE_SPREAD', 'LLMA2_ORIG_RATE_SPREAD_NAN', 
       'AGI', 'AGI_NAN', 'UR', 'UR_NAN', 'LLMA2_ORIG_RATE_ORIG_MR_SPREAD', 
       'LLMA2_ORIG_RATE_ORIG_MR_SPREAD_NAN', 'COUNT_INT_RATE_LESS', 'NUM_PRIME_ZIP', 'NUM_PRIME_ZIP_NAN'
       ]
ncols = [x for x in numeric_cols if x.find('NAN')<0]
nan_cols = [x for x in numeric_cols if x.find('NAN')>0]
print(len(ncols), ncols)   
print(len(nan_cols), nan_cols)   

47 ['MBA_DAYS_DELINQUENT', 'CURRENT_INTEREST_RATE', 'LOANAGE', 'CURRENT_BALANCE', 'SCHEDULED_PRINCIPAL', 'SCHEDULED_MONTHLY_PANDI', 'LLMA2_CURRENT_INTEREST_SPREAD', 'LLMA2_C_IN_LAST_12_MONTHS', 'LLMA2_30_IN_LAST_12_MONTHS', 'LLMA2_60_IN_LAST_12_MONTHS', 'LLMA2_90_IN_LAST_12_MONTHS', 'LLMA2_FC_IN_LAST_12_MONTHS', 'LLMA2_REO_IN_LAST_12_MONTHS', 'LLMA2_0_IN_LAST_12_MONTHS', 'LLMA2_HIST_LAST_12_MONTHS_MIS', 'NUM_MODIF', 'P_RATE_TO_MOD', 'MOD_RATE', 'DIF_RATE', 'P_MONTHLY_PAY', 'MOD_MONTHLY_PAY', 'DIF_MONTHLY_PAY', 'CAPITALIZATION_AMT', 'MORTGAGE_RATE', 'FICO_SCORE_ORIGINATION', 'INITIAL_INTEREST_RATE', 'ORIGINAL_LTV', 'ORIGINAL_BALANCE', 'BACKEND_RATIO', 'ORIGINAL_TERM', 'SALE_PRICE', 'PREPAY_PENALTY_TERM', 'NUMBER_OF_UNITS', 'MARGIN', 'PERIODIC_RATE_CAP', 'PERIODIC_RATE_FLOOR', 'LIFETIME_RATE_CAP', 'LIFETIME_RATE_FLOOR', 'RATE_RESET_FREQUENCY', 'PAY_RESET_FREQUENCY', 'FIRST_RATE_RESET_PERIOD', 'LLMA2_ORIG_RATE_SPREAD', 'AGI', 'UR', 'LLMA2_ORIG_RATE_ORIG_MR_SPREAD', 'COUNT_INT_RATE_LESS', 

#### Numerical Features

In [6]:
ALL_DATA = bd.get_h5_dataset(PRO_DIR, architecture, train_dir, valid_dir, None, train_period=train_period, valid_period=None, test_period=None, cols=None)
#memory error:
#ALL_DATA.train._dict[0]['dataset_features'] = np.array( ALL_DATA.train._dict[0]['dataset_features'])

/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/temporalloandynmodifmrstaticitur15mill-16mill-train_0.h5 ...to load
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/temporalloandynmodifmrstaticitur15mill-16mill-train_0.h5  loaded in RAM
class_weights [  445264   736172   259623   614496 24186991   499157    27885]
class_weights [0.983 0.972 0.99  0.977 0.096 0.981 0.999]
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_valid/temporalloandynmodifmrstaticitur15mill-16mill-valid_0.h5 ...to load
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_valid/temporalloandynmodifmrstaticitur15mill-16mill-valid_0.h5  loaded in RAM
class_weights [ 14560  19805   6681  15002 835450   9960    410]
class_weights [0.984 0.978 0.993 0.983 0.074 0.989 1.   ]


In [7]:
print(ALL_DATA.train._dict[0].keys())
print(len(ALL_DATA.train.features_list), ALL_DATA.train.features_list)
print(ALL_DATA.train._dict[0]['nrows'])

dict_keys(['dataset_features', 'dataset_labels', 'nrows', 'init_index', 'end_index', 'class_weights'])
258 ['MBA_DAYS_DELINQUENT', 'MBA_DAYS_DELINQUENT_NAN', 'CURRENT_INTEREST_RATE', 'CURRENT_INTEREST_RATE_NAN', 'LOANAGE', 'LOANAGE_NAN', 'CURRENT_BALANCE', 'CURRENT_BALANCE_NAN', 'SCHEDULED_PRINCIPAL', 'SCHEDULED_PRINCIPAL_NAN', 'SCHEDULED_MONTHLY_PANDI', 'SCHEDULED_MONTHLY_PANDI_NAN', 'LLMA2_CURRENT_INTEREST_SPREAD', 'LLMA2_CURRENT_INTEREST_SPREAD_NAN', 'LLMA2_C_IN_LAST_12_MONTHS', 'LLMA2_30_IN_LAST_12_MONTHS', 'LLMA2_60_IN_LAST_12_MONTHS', 'LLMA2_90_IN_LAST_12_MONTHS', 'LLMA2_FC_IN_LAST_12_MONTHS', 'LLMA2_REO_IN_LAST_12_MONTHS', 'LLMA2_0_IN_LAST_12_MONTHS', 'LLMA2_HIST_LAST_12_MONTHS_MIS', 'NUM_MODIF', 'NUM_MODIF_NAN', 'P_RATE_TO_MOD', 'P_RATE_TO_MOD_NAN', 'MOD_RATE', 'MOD_RATE_NAN', 'DIF_RATE', 'DIF_RATE_NAN', 'P_MONTHLY_PAY', 'P_MONTHLY_PAY_NAN', 'MOD_MONTHLY_PAY', 'MOD_MONTHLY_PAY_NAN', 'DIF_MONTHLY_PAY', 'DIF_MONTHLY_PAY_NAN', 'CAPITALIZATION_AMT', 'CAPITALIZATION_AMT_NAN', 'MORTG

In [8]:
#idx_ncols = [ALL_DATA.train.features_list.index(x) for x in ncols]
idx_nan_cols = [ALL_DATA.train.features_list.index(x) for x in nan_cols]
#print(len(idx_ncols), idx_ncols) #
print(len(idx_nan_cols), idx_nan_cols) #

34 [1, 3, 5, 7, 9, 11, 13, 23, 25, 27, 29, 31, 33, 35, 37, 39, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 74, 76, 78, 80, 83]


In [9]:
threshold = 0.5
dropping_cols = []
for i, col_name in zip(idx_nan_cols, nan_cols):    
    column = [elem[i] for elem in ALL_DATA.train._dict[0]['dataset_features']]
    perc_miss = np.average(np.array(column), axis=0)
    print('col_name: ', col_name, 'nan_index:', i, ' perc_miss: ', perc_miss)
    if perc_miss >= threshold:
        k = col_name[:-4]
        i_ncol = ALL_DATA.train.features_list.index(k)
        print('i_ncol: ', i_ncol)
        dropping_cols.extend([i, i_ncol])

col_name:  MBA_DAYS_DELINQUENT_NAN nan_index: 1  perc_miss:  0.0010629973
col_name:  CURRENT_INTEREST_RATE_NAN nan_index: 3  perc_miss:  0.0053772214
col_name:  LOANAGE_NAN nan_index: 5  perc_miss:  0.00019171009
col_name:  CURRENT_BALANCE_NAN nan_index: 7  perc_miss:  1.0086072e-06
col_name:  SCHEDULED_PRINCIPAL_NAN nan_index: 9  perc_miss:  0.50619245
i_ncol:  8
col_name:  SCHEDULED_MONTHLY_PANDI_NAN nan_index: 11  perc_miss:  0.07452367
col_name:  LLMA2_CURRENT_INTEREST_SPREAD_NAN nan_index: 13  perc_miss:  0.0053772214
col_name:  NUM_MODIF_NAN nan_index: 23  perc_miss:  0.94826007
i_ncol:  22
col_name:  P_RATE_TO_MOD_NAN nan_index: 25  perc_miss:  0.94826007
i_ncol:  24
col_name:  MOD_RATE_NAN nan_index: 27  perc_miss:  0.94826007
i_ncol:  26
col_name:  DIF_RATE_NAN nan_index: 29  perc_miss:  0.94826007
i_ncol:  28
col_name:  P_MONTHLY_PAY_NAN nan_index: 31  perc_miss:  0.94826007
i_ncol:  30
col_name:  MOD_MONTHLY_PAY_NAN nan_index: 33  perc_miss:  0.94826007
i_ncol:  32
col_name:

In [10]:
print(len(dropping_cols), dropping_cols)

40 [9, 8, 23, 22, 25, 24, 27, 26, 29, 28, 31, 30, 33, 32, 35, 34, 37, 36, 45, 44, 49, 48, 51, 50, 55, 54, 57, 56, 59, 58, 61, 60, 63, 62, 65, 64, 67, 66, 69, 68]


#### Transforming and Saving Dataset

In [11]:
for v in dropping_cols:
    del ALL_DATA.train.features_list[v]        

In [12]:
print(len(ALL_DATA.train.features_list), ALL_DATA.train.features_list)
print(len(ALL_DATA.train.labels_list), ALL_DATA.train.labels_list)
pd.set_option('io.hdf.default_format','table')

218 ['MBA_DAYS_DELINQUENT', 'MBA_DAYS_DELINQUENT_NAN', 'CURRENT_INTEREST_RATE', 'CURRENT_INTEREST_RATE_NAN', 'LOANAGE', 'LOANAGE_NAN', 'CURRENT_BALANCE', 'CURRENT_BALANCE_NAN', 'SCHEDULED_MONTHLY_PANDI', 'SCHEDULED_MONTHLY_PANDI_NAN', 'LLMA2_CURRENT_INTEREST_SPREAD', 'LLMA2_CURRENT_INTEREST_SPREAD_NAN', 'LLMA2_C_IN_LAST_12_MONTHS', 'LLMA2_30_IN_LAST_12_MONTHS', 'LLMA2_60_IN_LAST_12_MONTHS', 'LLMA2_90_IN_LAST_12_MONTHS', 'LLMA2_FC_IN_LAST_12_MONTHS', 'LLMA2_REO_IN_LAST_12_MONTHS', 'LLMA2_0_IN_LAST_12_MONTHS', 'LLMA2_HIST_LAST_12_MONTHS_MIS', 'NUM_MODIF', 'NUM_MODIF_NAN', 'MOD_RATE', 'MOD_RATE_NAN', 'P_MONTHLY_PAY', 'P_MONTHLY_PAY_NAN', 'DIF_MONTHLY_PAY', 'DIF_MONTHLY_PAY_NAN', 'MORTGAGE_RATE', 'MORTGAGE_RATE_NAN', 'ORIGINAL_LTV', 'ORIGINAL_BALANCE', 'ORIGINAL_TERM', 'ORIGINAL_TERM_NAN', 'PREPAY_PENALTY_TERM', 'PREPAY_PENALTY_TERM_NAN', 'MARGIN', 'MARGIN_NAN', 'PERIODIC_RATE_CAP', 'PERIODIC_RATE_CAP_NAN', 'PERIODIC_RATE_FLOOR', 'PERIODIC_RATE_FLOOR_NAN', 'LIFETIME_RATE_CAP', 'LIFETIME_RA

In [18]:
# Saving into another .h5 dataset:
def missing_remove_save(tag, file_dir, data_dict, features_list, labels_list, drop_cols, batch_size): # ALL_DATA.train._dict
    file_name = 'dataset_missratio_'+ tag + '_.h5'
    target_path = os.path.join(PRO_DIR, file_dir, file_name)        
    total_rows = data_dict[0]['nrows']
    with pd.HDFStore(target_path, complib='lzo', complevel=9) as hdf_target:
        print('Target Path: ', target_path)   
        # idxs = np.resize(drop_cols.values(),np.count_nonzero(drop_cols.values()))                
        # print('columns dropped')
        num_iter = math.ceil(np.float32(total_rows / batch_size))
        acc_rows = 0
        for i in range(num_iter):
            print('batch: ', i)    
            if ((i+1)*batch_size > total_rows):            
                feat_batch = data_dict[0]['dataset_features'][i*batch_size : ]     
                lab_batch = data_dict[0]['dataset_labels'][i*batch_size : ]     
            else :            
                feat_batch = data_dict[0]['dataset_features'][i*batch_size : (i+1)*batch_size]
                lab_batch = data_dict[0]['dataset_labels'][i*batch_size : (i+1)*batch_size]     
            #data_dict[0]['dataset_features'] = np.delete(data_dict[0]['dataset_features'], drop_cols, axis=1)            
            df_feat = pd.DataFrame(feat_batch) #columns=features_list
            df_feat.drop(drop_cols, axis=1, inplace=True)
            df_feat.columns = features_list
            print('drop_cols dropped')
            df_feat = df_feat.astype(DT_FLOAT)
            #print(df_feat[:2])
            #print(df_feat.shape)                    
            df_lab = pd.DataFrame(lab_batch, columns=labels_list)
            df_lab = df_lab.astype(np.int8)
            hdf_target.put(tag + '/features', df_feat, append=True) 
            hdf_target.put(tag + '/labels', df_lab, append=True) 
            hdf_target.flush()
            print('batch saved')    
            acc_rows += df_feat.shape[0]
        print('total_rows: ', total_rows)
        print('acc_rows: ', acc_rows)


In [19]:
missing_remove_save('train', train_dir, ALL_DATA.train._dict, ALL_DATA.train.features_list, ALL_DATA.train.labels_list, dropping_cols, 150000)

Target Path:  /home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/dataset_missratio_train_.h5
batch:  0
drop_cols dropped
batch saved
batch:  1
drop_cols dropped
batch saved
batch:  2
drop_cols dropped
batch saved
batch:  3
drop_cols dropped
batch saved
batch:  4
drop_cols dropped
batch saved
batch:  5
drop_cols dropped
batch saved
batch:  6
drop_cols dropped
batch saved
batch:  7
drop_cols dropped
batch saved
batch:  8
drop_cols dropped
batch saved
batch:  9
drop_cols dropped
batch saved
batch:  10
drop_cols dropped
batch saved
batch:  11
drop_cols dropped
batch saved
batch:  12
drop_cols dropped
batch saved
batch:  13
drop_cols dropped
batch saved
batch:  14
drop_cols dropped
batch saved
batch:  15
drop_cols dropped
batch saved
batch:  16
drop_cols dropped
batch saved
batch:  17
drop_cols dropped
batch saved
batch:  18
drop_cols dropped
batch saved
batch:  19
drop_cols dropped
batch saved
batch:  20
drop_cols dropped
batch saved
batch:  21
drop_cols dropped
batch sav

In [20]:
missing_remove_save('valid', valid_dir, ALL_DATA.validation._dict, ALL_DATA.train.features_list, ALL_DATA.train.labels_list, dropping_cols, 150000)

Target Path:  /home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_valid/dataset_missratio_valid_.h5
batch:  0
drop_cols dropped
batch saved
batch:  1
drop_cols dropped
batch saved
batch:  2
drop_cols dropped
batch saved
batch:  3
drop_cols dropped
batch saved
batch:  4
drop_cols dropped
batch saved
batch:  5
drop_cols dropped
batch saved
batch:  6
drop_cols dropped
batch saved
total_rows:  901868
acc_rows:  901868
